In [21]:
# city_centers_osm_cities_landmark_g_business_polygon_dbscan_v3.py
# Historic center g: geocoded landmark per city
# Business area: DBSCAN cluster of OSM business POIs + buffered-union polygon
# Business center mu: centroid of business polygon
# Outputs: distance (m), business area (sq km), and saves polygon shapes
#
# Updates:
# - Removed these cities completely: Rabat, New Delhi, Cairo, Islamabad, Valletta, Jerusalem
# - Moscow-only settings fixed to:
#   DBSCAN_EPS_M = 320, DBSCAN_MIN_SAMPLES = 60
#   POLY_POINT_BUFFER_M = 140, POLY_SIMPLIFY_M = 80

import os
import re
import math
import time
import random
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox

from shapely.geometry import Point
from shapely.ops import unary_union

warnings.filterwarnings("ignore", category=UserWarning)

# -----------------------------
# Output paths
# -----------------------------
OUT_DIR = "data/city_centers"
os.makedirs(OUT_DIR, exist_ok=True)

POLY_DIR = os.path.join(OUT_DIR, "business_area_polygons_geojson")
os.makedirs(POLY_DIR, exist_ok=True)

OUT_CSV = os.path.join(OUT_DIR, "city_centers_business_polygon.csv")
OUT_GPKG = os.path.join(OUT_DIR, "business_areas.gpkg")
OUT_GEOJSON_ALL = os.path.join(OUT_DIR, "business_areas.geojson")

RESUME_IF_EXISTS = True

# -----------------------------
# City core settings
# -----------------------------
MAX_RADIUS_KM_DEFAULT = 12  # core radius around geocoded city center

# Overpass load controls for very large metros
CITY_OVERRIDES = {
    "New York City, NY, USA": {"radius_km": 10},
    "London, UK": {"radius_km": 10},
    "Moscow, Russia": {"radius_km": 10},
    "Tokyo, Japan": {"radius_km": 10},
    "Shanghai, China": {"radius_km": 10},
    "Los Angeles, CA, USA": {"radius_km": 10},
    "Mexico City, Mexico": {"radius_km": 10},
    "São Paulo, Brazil": {"radius_km": 10},
    "Rio de Janeiro, Brazil": {"radius_km": 10},
}

# -----------------------------
# Business POI tags
# -----------------------------
INCLUDE_SHOPS = True  # if timeouts, set False

def business_tags(include_shops: bool):
    tags = {
        "office": True,
        "building": ["office", "commercial"],
        "landuse": ["commercial", "retail"],
        "amenity": ["bank", "conference_centre"],
    }
    if include_shops:
        tags["shop"] = True
    return tags

# -----------------------------
# DBSCAN defaults (meters)
# -----------------------------
DBSCAN_EPS_M_DEFAULT = 450
DBSCAN_MIN_SAMPLES_DEFAULT = 25

# Polygon build defaults (meters)
POLY_POINT_BUFFER_M_DEFAULT = 180
POLY_SIMPLIFY_M_DEFAULT = 60

MIN_POINTS_FOR_CLUSTERING = 80  # if fewer points than this, skip clustering and polygonize all points

# Moscow-only override (exactly as requested)
CITY_CLUSTER_OVERRIDES = {
    "Moscow, Russia": {
        "eps_m": 420,
        "min_samples": 60,
        "point_buffer_m": 140,
        "simplify_m": 80,
    }
}

# Politeness + retries
PAUSE_BETWEEN_CITIES_S = 1.0
TRIES = 4
BASE_SLEEP = 2.0
JITTER = 0.7

# -----------------------------
# Cities (removed: Rabat, New Delhi, Cairo, Islamabad, Valletta, Jerusalem)
# -----------------------------
CITIES = [
    # Europe
    "Paris, France",
    "London, UK",
    "Berlin, Germany",
    "Rome, Italy",
    "Madrid, Spain",
    "Lisbon, Portugal",
    "Amsterdam, Netherlands",
    "Brussels, Belgium",
    "Copenhagen, Denmark",
    "Stockholm, Sweden",
    "Oslo, Norway",
    "Helsinki, Finland",
    "Dublin, Ireland",
    "Reykjavik, Iceland",
    "Vienna, Austria",
    "Prague, Czechia",
    "Warsaw, Poland",
    "Budapest, Hungary",
    "Bratislava, Slovakia",
    "Ljubljana, Slovenia",
    "Zagreb, Croatia",
    "Belgrade, Serbia",
    "Sarajevo, Bosnia and Herzegovina",
    "Podgorica, Montenegro",
    "Skopje, North Macedonia",
    "Tirana, Albania",
    "Nicosia, Cyprus",
    "Sofia, Bulgaria",
    "Bucharest, Romania",
    "Chișinău, Moldova",
    "Kyiv, Ukraine",
    "Minsk, Belarus",
    "Moscow, Russia",
    "Istanbul, Turkey",
    "Ankara, Turkey",
    "Tbilisi, Georgia",
    "Yerevan, Armenia",
    "Baku, Azerbaijan",

    # Middle East / North Africa (Cairo + Rabat removed, Jerusalem removed)
    "Amman, Jordan",
    "Beirut, Lebanon",
    "Doha, Qatar",
    "Abu Dhabi, United Arab Emirates",
    "Dubai, United Arab Emirates",
    "Tunis, Tunisia",
    "Manama, Bahrain",
    "Tehran, Iran",
    "Baghdad, Iraq",

    # South Asia (New Delhi + Islamabad removed)
    "Bengaluru, India",
    "Dhaka, Bangladesh",
    "Colombo, Sri Lanka",

    # Southeast Asia
    "Bangkok, Thailand",
    "Hanoi, Vietnam",
    "Ho Chi Minh City, Vietnam",
    "Kuala Lumpur, Malaysia",
    "Singapore",
    "Jakarta, Indonesia",
    "Manila, Philippines",
    "Phnom Penh, Cambodia",
    "Vientiane, Laos",
    "Yangon, Myanmar",

    # East Asia
    "Hong Kong",
    "Shanghai, China",
    "Tokyo, Japan",
    "Seoul, South Korea",
    "Taipei, Taiwan",

    # Central Asia
    "Tashkent, Uzbekistan",
    "Astana, Kazakhstan",

    # Oceania
    "Canberra, Australia",
    "Sydney, Australia",
    "Wellington, New Zealand",
    "Auckland, New Zealand",

    # North America
    "Washington, DC, USA",
    "New York City, NY, USA",
    "Chicago, IL, USA",
    "Los Angeles, CA, USA",
    "San Francisco, CA, USA",
    "Ottawa, Canada",
    "Toronto, Canada",
    "Montreal, Canada",
    "Mexico City, Mexico",
    "Havana, Cuba",
    "Guatemala City, Guatemala",

    # South America / Caribbean
    "Bogotá, Colombia",
    "Caracas, Venezuela",
    "Quito, Ecuador",
    "Lima, Peru",
    "La Paz, Bolivia",
    "Santiago, Chile",
    "Buenos Aires, Argentina",
    "Montevideo, Uruguay",
    "Asunción, Paraguay",
    "Santo Domingo, Dominican Republic",
    "Rio de Janeiro, Brazil",
    "São Paulo, Brazil",
]

# -----------------------------
# Historic landmark queries (removed the ones for deleted cities)
# -----------------------------
HISTORIC_CENTER_QUERIES = {
    # Europe
    "Paris, France": "Notre-Dame Cathedral, Paris, France",
    "London, UK": "St Paul's Cathedral, London, UK",
    "Berlin, Germany": "Brandenburg Gate, Berlin, Germany",
    "Rome, Italy": "Pantheon, Rome, Italy",
    "Madrid, Spain": "Puerta del Sol, Madrid, Spain",
    "Lisbon, Portugal": "Praça do Comércio, Lisbon, Portugal",
    "Amsterdam, Netherlands": "Dam Square, Amsterdam, Netherlands",
    "Brussels, Belgium": "Grand-Place, Brussels, Belgium",
    "Copenhagen, Denmark": "Christiansborg Palace, Copenhagen, Denmark",
    "Stockholm, Sweden": "Stortorget, Stockholm, Sweden",
    "Oslo, Norway": "Akershus Fortress, Oslo, Norway",
    "Helsinki, Finland": "Senate Square, Helsinki, Finland",
    "Dublin, Ireland": "Dublin Castle, Dublin, Ireland",
    "Reykjavik, Iceland": "Austurvöllur, Reykjavík, Iceland",
    "Vienna, Austria": "Stephansplatz, Vienna, Austria",
    "Prague, Czechia": "Old Town Square, Prague, Czechia",
    "Warsaw, Poland": "Castle Square, Warsaw, Poland",
    "Budapest, Hungary": "Buda Castle, Budapest, Hungary",
    "Bratislava, Slovakia": "Bratislava Castle, Bratislava, Slovakia",
    "Ljubljana, Slovenia": "Ljubljana Castle, Ljubljana, Slovenia",
    "Zagreb, Croatia": "Ban Jelačić Square, Zagreb, Croatia",
    "Belgrade, Serbia": "Kalemegdan Fortress, Belgrade, Serbia",
    "Sarajevo, Bosnia and Herzegovina": "Baščaršija, Sarajevo",
    "Podgorica, Montenegro": "Stara Varoš, Podgorica, Montenegro",
    "Skopje, North Macedonia": "Stone Bridge, Skopje",
    "Tirana, Albania": "Skanderbeg Square, Tirana, Albania",
    "Nicosia, Cyprus": "Ledra Street, Nicosia, Cyprus",
    "Sofia, Bulgaria": "Alexander Nevsky Cathedral, Sofia, Bulgaria",
    "Bucharest, Romania": "University Square, Bucharest, Romania",
    "Chișinău, Moldova": "Cathedral Park, Chișinău, Moldova",
    "Kyiv, Ukraine": "Maidan Nezalezhnosti, Kyiv, Ukraine",
    "Minsk, Belarus": "Independence Square, Minsk, Belarus",
    "Moscow, Russia": "Red Square, Moscow, Russia",
    "Istanbul, Turkey": "Sultanahmet Square, Istanbul, Turkey",
    "Ankara, Turkey": "Ulus Square, Ankara, Turkey",
    "Tbilisi, Georgia": "Freedom Square, Tbilisi, Georgia",
    "Yerevan, Armenia": "Republic Square, Yerevan, Armenia",
    "Baku, Azerbaijan": "Icherisheher, Baku, Azerbaijan",

    # Middle East / North Africa
    "Amman, Jordan": "Amman Citadel, Amman, Jordan",
    "Beirut, Lebanon": "Martyrs' Square, Beirut, Lebanon",
    "Doha, Qatar": "Souq Waqif, Doha, Qatar",
    "Abu Dhabi, United Arab Emirates": "Qasr Al Hosn, Abu Dhabi, UAE",
    "Dubai, United Arab Emirates": "Al Fahidi Historical Neighbourhood, Dubai, UAE",
    "Tunis, Tunisia": "Medina of Tunis, Tunis, Tunisia",
    "Manama, Bahrain": "Bab Al Bahrain, Manama, Bahrain",
    "Tehran, Iran": "Golestan Palace, Tehran, Iran",
    "Baghdad, Iraq": "Al-Mutanabbi Street, Baghdad, Iraq",

    # South Asia
    "Bengaluru, India": "Bangalore Fort, Bengaluru, India",
    "Dhaka, Bangladesh": "Lalbagh Fort, Dhaka, Bangladesh",
    "Colombo, Sri Lanka": "Colombo Fort, Colombo, Sri Lanka",

    # Southeast Asia
    "Bangkok, Thailand": "Grand Palace, Bangkok, Thailand",
    "Hanoi, Vietnam": "Hoàn Kiếm Lake, Hanoi, Vietnam",
    "Ho Chi Minh City, Vietnam": "Bến Thành Market, Ho Chi Minh City, Vietnam",
    "Kuala Lumpur, Malaysia": "Merdeka Square, Kuala Lumpur, Malaysia",
    "Singapore": "City Hall, Singapore",
    "Jakarta, Indonesia": "Monas, Jakarta, Indonesia",
    "Manila, Philippines": "Intramuros, Manila, Philippines",
    "Phnom Penh, Cambodia": "Royal Palace, Phnom Penh, Cambodia",
    "Vientiane, Laos": "That Luang, Vientiane, Laos",
    "Yangon, Myanmar": "Sule Pagoda, Yangon, Myanmar",

    # East Asia
    "Hong Kong": "Man Mo Temple, Hong Kong",
    "Shanghai, China": "The Bund, Shanghai, China",
    "Tokyo, Japan": "Imperial Palace, Tokyo, Japan",
    "Seoul, South Korea": "Gyeongbokgung Palace, Seoul, South Korea",
    "Taipei, Taiwan": "Chiang Kai-shek Memorial Hall, Taipei, Taiwan",

    # Central Asia
    "Tashkent, Uzbekistan": "Khast Imam Complex, Tashkent, Uzbekistan",
    "Astana, Kazakhstan": "Bayterek Tower, Astana, Kazakhstan",

    # Oceania
    "Canberra, Australia": "Old Parliament House, Canberra, Australia",
    "Sydney, Australia": "The Rocks, Sydney, Australia",
    "Wellington, New Zealand": "Old St Paul's, Wellington, New Zealand",
    "Auckland, New Zealand": "Auckland Town Hall, Auckland, New Zealand",

    # North America
    "Washington, DC, USA": "United States Capitol, Washington, DC, USA",
    "New York City, NY, USA": "Federal Hall, New York, NY, USA",
    "Chicago, IL, USA": "The Loop, Chicago, IL, USA",
    "Los Angeles, CA, USA": "El Pueblo de Los Angeles, Los Angeles, CA, USA",
    "San Francisco, CA, USA": "Portsmouth Square, San Francisco, CA, USA",
    "Ottawa, Canada": "Parliament Hill, Ottawa, Canada",
    "Toronto, Canada": "Old City Hall, Toronto, Canada",
    "Montreal, Canada": "Notre-Dame Basilica, Montreal, Canada",
    "Mexico City, Mexico": "Zócalo, Mexico City, Mexico",
    "Havana, Cuba": "Plaza de Armas, Havana, Cuba",
    "Guatemala City, Guatemala": "Plaza de la Constitución, Guatemala City, Guatemala",

    # South America / Caribbean
    "Bogotá, Colombia": "Plaza de Bolívar, Bogotá, Colombia",
    "Caracas, Venezuela": "Plaza Bolívar, Caracas, Venezuela",
    "Quito, Ecuador": "Plaza Grande, Quito, Ecuador",
    "Lima, Peru": "Plaza Mayor, Lima, Peru",
    "La Paz, Bolivia": "Plaza Murillo, La Paz, Bolivia",
    "Santiago, Chile": "Plaza de Armas, Santiago, Chile",
    "Buenos Aires, Argentina": "Plaza de Mayo, Buenos Aires, Argentina",
    "Montevideo, Uruguay": "Plaza Independencia, Montevideo, Uruguay",
    "Asunción, Paraguay": "Plaza de Armas, Asunción, Paraguay",
    "Santo Domingo, Dominican Republic": "Zona Colonial, Santo Domingo, Dominican Republic",
    "Rio de Janeiro, Brazil": "Paço Imperial, Rio de Janeiro, Brazil",
    "São Paulo, Brazil": "Pateo do Collegio, São Paulo, Brazil",
}

# -----------------------------
# Helpers
# -----------------------------
def configure_osmnx():
    ox.settings.use_cache = True
    ox.settings.log_console = True
    ox.settings.timeout = 180
    if hasattr(ox.settings, "overpass_rate_limit"):
        ox.settings.overpass_rate_limit = True

def with_retries(fn, *args, tries=TRIES, base_sleep=BASE_SLEEP, jitter=JITTER, **kwargs):
    last = None
    for k in range(tries):
        try:
            return fn(*args, **kwargs)
        except Exception as e:
            last = e
            time.sleep(base_sleep * (2 ** k) + random.random() * jitter)
    raise last

def slugify(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[^a-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s[:120]

def safe_geocode_point(query: str) -> Point:
    lat, lon = with_retries(ox.geocode, query)  # returns (lat, lon)
    return Point(float(lon), float(lat))

def get_city_boundary_ll(city_query: str) -> gpd.GeoDataFrame:
    gdf = with_retries(ox.geocode_to_gdf, city_query)
    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
    if gdf.empty:
        raise RuntimeError(f"Could not fetch polygon for: {city_query}")
    geom = gdf.geometry.iloc[0]
    return gpd.GeoDataFrame({"name": [city_query]}, geometry=[geom], crs=gdf.crs).to_crs("EPSG:4326")

def utm_epsg_from_lonlat(lon: float, lat: float) -> str:
    zone = int((lon + 180.0) // 6) + 1
    epsg = (32600 + zone) if (lat >= 0) else (32700 + zone)
    return f"EPSG:{epsg}"

def choose_local_utm_crs(boundary_ll: gpd.GeoDataFrame) -> str:
    try:
        crs = boundary_ll.estimate_utm_crs()
        if crs is not None:
            return crs.to_string()
    except Exception:
        pass
    c = boundary_ll.geometry.iloc[0].centroid
    return utm_epsg_from_lonlat(float(c.x), float(c.y))

def project_point_ll_to_utm(lon: float, lat: float, utm_crs: str) -> Point:
    return (
        gpd.GeoDataFrame(geometry=[Point(lon, lat)], crs="EPSG:4326")
        .to_crs(utm_crs)
        .geometry.iloc[0]
    )

def xy_to_lonlat(x: float, y: float, utm_crs: str):
    pt = gpd.GeoDataFrame(geometry=[Point(x, y)], crs=utm_crs).to_crs("EPSG:4326").geometry.iloc[0]
    return float(pt.x), float(pt.y)

def make_core_analysis_polygon(boundary_ll: gpd.GeoDataFrame, city_center_ll_point: Point, max_radius_km: float):
    utm_crs = choose_local_utm_crs(boundary_ll)

    boundary_utm = boundary_ll.to_crs(utm_crs)
    center_utm = (
        gpd.GeoDataFrame(geometry=[city_center_ll_point], crs="EPSG:4326")
        .to_crs(utm_crs)
        .geometry.iloc[0]
    )

    core_circle = center_utm.buffer(max_radius_km * 1000.0)
    boundary_geom_utm = boundary_utm.geometry.iloc[0]
    core_utm_geom = boundary_geom_utm.intersection(core_circle)
    if core_utm_geom.is_empty:
        core_utm_geom = core_circle

    core_utm_gdf = gpd.GeoDataFrame({"name": ["core"]}, geometry=[core_utm_geom], crs=utm_crs)
    core_ll_gdf = core_utm_gdf.to_crs("EPSG:4326")
    return core_ll_gdf, core_utm_gdf, utm_crs

def fetch_osm_features(poly_ll, tags: dict) -> gpd.GeoDataFrame:
    def _fetch():
        try:
            return ox.features_from_polygon(poly_ll, tags)
        except Exception:
            return ox.geometries_from_polygon(poly_ll, tags)

    gdf = with_retries(_fetch)
    if gdf is None or len(gdf) == 0:
        return gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326")
    else:
        gdf = gdf.to_crs("EPSG:4326")
    return gdf

def features_to_points_ll(features_ll: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    if features_ll.empty:
        return gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

    def to_pt(g):
        if g is None:
            return None
        if g.geom_type == "Point":
            return g
        try:
            return g.representative_point()
        except Exception:
            return g.centroid

    pts = features_ll.copy()
    pts["geometry"] = pts.geometry.apply(to_pt)
    pts = pts[pts.geometry.notna()].copy()
    pts = pts.set_geometry("geometry")
    pts = pts.set_crs("EPSG:4326", allow_override=True)
    pts = pts[pts.geometry.type == "Point"].copy()
    return pts[["geometry"]].copy()

def largest_polygon(geom):
    if geom is None or geom.is_empty:
        return None
    if geom.geom_type == "Polygon":
        return geom
    if geom.geom_type == "MultiPolygon":
        return max(list(geom.geoms), key=lambda g: g.area)
    return None

def get_historic_center_ll(city_query: str):
    tried = []
    q = HISTORIC_CENTER_QUERIES.get(city_query, None)
    if q:
        tried.append(q)

    tried += [
        f"Old Town, {city_query}",
        f"Historic Centre, {city_query}",
        f"City Centre, {city_query}",
        f"Central Square, {city_query}",
        city_query,
    ]

    last_err = None
    for qq in tried:
        try:
            p = safe_geocode_point(qq)
            return float(p.x), float(p.y), f"geocode:{qq}"
        except Exception as e:
            last_err = e

    raise RuntimeError(f"Could not geocode historic center for {city_query}. Last error: {last_err}")

# -----------------------------
# DBSCAN + polygon
# -----------------------------
def run_dbscan(points_xy: np.ndarray, eps_m: float, min_samples: int):
    try:
        from sklearn.cluster import DBSCAN
    except Exception:
        return None
    model = DBSCAN(eps=float(eps_m), min_samples=int(min_samples))
    return model.fit_predict(points_xy)

def get_city_cluster_params(city: str):
    ov = CITY_CLUSTER_OVERRIDES.get(city, {})
    eps_m = float(ov.get("eps_m", DBSCAN_EPS_M_DEFAULT))
    min_samples = int(ov.get("min_samples", DBSCAN_MIN_SAMPLES_DEFAULT))
    point_buffer_m = float(ov.get("point_buffer_m", POLY_POINT_BUFFER_M_DEFAULT))
    simplify_m = float(ov.get("simplify_m", POLY_SIMPLIFY_M_DEFAULT))
    return eps_m, min_samples, point_buffer_m, simplify_m

def pick_main_cluster(points_utm: gpd.GeoDataFrame, eps_m: float, min_samples: int):
    n = len(points_utm)
    if n == 0:
        return points_utm, "no_points", 0

    if n < MIN_POINTS_FOR_CLUSTERING:
        return points_utm, "no_cluster_small_n", n

    xy = np.column_stack([points_utm.geometry.x.values, points_utm.geometry.y.values])

    labels = run_dbscan(xy, eps_m, min_samples)
    if labels is None:
        return points_utm, "no_sklearn_fallback_all_points", n

    valid = labels[labels >= 0]
    if len(valid) == 0:
        return points_utm, "dbscan_all_noise_fallback_all_points", n

    uniq, cnt = np.unique(valid, return_counts=True)
    best_label = int(uniq[np.argmax(cnt)])

    cluster_pts = points_utm.iloc[labels == best_label].copy()
    return cluster_pts, f"dbscan_eps{eps_m}_min{min_samples}_label{best_label}", int(len(cluster_pts))

def business_polygon_from_points(points_cluster_utm: gpd.GeoDataFrame, point_buffer_m: float, simplify_m: float):
    if points_cluster_utm.empty:
        return None, 0.0

    bufs = points_cluster_utm.geometry.buffer(point_buffer_m)
    merged = unary_union(bufs.values)
    poly = largest_polygon(merged)

    if poly is None or poly.is_empty:
        return None, 0.0

    if simplify_m and simplify_m > 0:
        try:
            poly = poly.simplify(simplify_m, preserve_topology=True)
        except Exception:
            pass

    return poly, float(poly.area) / 1e6

# -----------------------------
# Per-city pipeline
# -----------------------------
def compute_city(city_query: str):
    cfg = CITY_OVERRIDES.get(city_query, {})
    radius_km = float(cfg.get("radius_km", MAX_RADIUS_KM_DEFAULT))

    eps_m, min_samples, point_buffer_m, simplify_m = get_city_cluster_params(city_query)

    print(f"\nProcessing: {city_query}")
    print(f"  core radius km: {radius_km}")
    print(f"  dbscan eps/min_samples: {eps_m}/{min_samples}")
    print(f"  polygon buffer/simplify (m): {point_buffer_m}/{simplify_m}")

    boundary_ll = get_city_boundary_ll(city_query)
    city_center_ll = safe_geocode_point(city_query)

    core_ll_gdf, _, utm_crs = make_core_analysis_polygon(boundary_ll, city_center_ll, radius_km)
    core_poly_ll = core_ll_gdf.geometry.iloc[0]

    tags = business_tags(INCLUDE_SHOPS)
    bus_raw_ll = fetch_osm_features(core_poly_ll, tags)
    bus_pts_ll = features_to_points_ll(bus_raw_ll)
    bus_pts_utm = bus_pts_ll.to_crs(utm_crs) if not bus_pts_ll.empty else bus_pts_ll

    g_lon, g_lat, _ = get_historic_center_ll(city_query)
    g_pt_utm = project_point_ll_to_utm(g_lon, g_lat, utm_crs)
    gx, gy = float(g_pt_utm.x), float(g_pt_utm.y)

    cluster_pts, cluster_method, cluster_n = pick_main_cluster(bus_pts_utm, eps_m=eps_m, min_samples=min_samples)
    poly_utm, area_km2 = business_polygon_from_points(cluster_pts, point_buffer_m=point_buffer_m, simplify_m=simplify_m)

    if poly_utm is None:
        mu_lon, mu_lat = float(city_center_ll.x), float(city_center_ll.y)
        mu_pt_utm = project_point_ll_to_utm(mu_lon, mu_lat, utm_crs)
        mux, muy = float(mu_pt_utm.x), float(mu_pt_utm.y)
        dist_m = float(math.hypot(mux - gx, muy - gy))

        row = {
            "city": city_query,
            "historic_lon": g_lon, "historic_lat": g_lat,
            "business_lon": mu_lon, "business_lat": mu_lat,
            "dist_hist_to_bus_m": dist_m,
            "business_area_km2": 0.0,
            "business_points_total": int(len(bus_pts_ll)),
            "business_points_cluster": int(cluster_n),
            "cluster_method": cluster_method,
            "business_center_method": "fallback_city_center_no_polygon",
            "polygon_file": "",
            "utm_crs": utm_crs,
            "core_radius_km": radius_km,
            "include_shops": bool(INCLUDE_SHOPS),
            "dbscan_eps_m": eps_m,
            "dbscan_min_samples": min_samples,
            "poly_point_buffer_m": point_buffer_m,
            "poly_simplify_m": simplify_m,
        }
        return row, None

    mu_cent = poly_utm.centroid
    mux, muy = float(mu_cent.x), float(mu_cent.y)
    mu_lon, mu_lat = xy_to_lonlat(mux, muy, utm_crs)
    dist_m = float(math.hypot(mux - gx, muy - gy))

    poly_ll = gpd.GeoDataFrame(
        {
            "city": [city_query],
            "area_km2": [float(area_km2)],
            "cluster_method": [cluster_method],
            "business_points_total": [int(len(bus_pts_ll))],
            "business_points_cluster": [int(cluster_n)],
            "dbscan_eps_m": [eps_m],
            "dbscan_min_samples": [min_samples],
            "poly_point_buffer_m": [point_buffer_m],
            "poly_simplify_m": [simplify_m],
        },
        geometry=[poly_utm],
        crs=utm_crs,
    ).to_crs("EPSG:4326")

    poly_path = os.path.join(POLY_DIR, f"{slugify(city_query)}.geojson")
    poly_ll.to_file(poly_path, driver="GeoJSON")

    row = {
        "city": city_query,
        "historic_lon": g_lon, "historic_lat": g_lat,
        "business_lon": mu_lon, "business_lat": mu_lat,
        "dist_hist_to_bus_m": dist_m,
        "business_area_km2": float(area_km2),
        "business_points_total": int(len(bus_pts_ll)),
        "business_points_cluster": int(cluster_n),
        "cluster_method": cluster_method,
        "business_center_method": "polygon_centroid",
        "polygon_file": poly_path,
        "utm_crs": utm_crs,
        "core_radius_km": radius_km,
        "include_shops": bool(INCLUDE_SHOPS),
        "dbscan_eps_m": eps_m,
        "dbscan_min_samples": min_samples,
        "poly_point_buffer_m": point_buffer_m,
        "poly_simplify_m": simplify_m,
    }
    return row, poly_ll

# -----------------------------
# Main
# -----------------------------
def main():
    configure_osmnx()

    rows = []
    polys = []

    done = set()
    if RESUME_IF_EXISTS and os.path.exists(OUT_CSV) and os.path.getsize(OUT_CSV) > 0:
        prev = pd.read_csv(OUT_CSV)
        if "city" in prev.columns:
            done = set(prev["city"].astype(str).tolist())
            rows = prev.to_dict(orient="records")
            print(f"Resuming from {OUT_CSV}. Cities already done: {len(done)}")

    if RESUME_IF_EXISTS and done:
        for city in done:
            fp = os.path.join(POLY_DIR, f"{slugify(city)}.geojson")
            if os.path.exists(fp) and os.path.getsize(fp) > 0:
                try:
                    polys.append(gpd.read_file(fp))
                except Exception:
                    pass

    for city in CITIES:
        if city in done:
            continue

        try:
            row, poly_ll = compute_city(city)
        except Exception as e:
            print(f"  Error for {city}: {e}")
            cfg = CITY_OVERRIDES.get(city, {})
            eps_m, min_samples, point_buffer_m, simplify_m = get_city_cluster_params(city)
            row = {
                "city": city,
                "historic_lon": np.nan, "historic_lat": np.nan,
                "business_lon": np.nan, "business_lat": np.nan,
                "dist_hist_to_bus_m": np.nan,
                "business_area_km2": np.nan,
                "business_points_total": 0,
                "business_points_cluster": 0,
                "cluster_method": f"error:{type(e).__name__}",
                "business_center_method": f"error:{type(e).__name__}",
                "polygon_file": "",
                "utm_crs": "",
                "core_radius_km": float(cfg.get("radius_km", MAX_RADIUS_KM_DEFAULT)),
                "include_shops": bool(INCLUDE_SHOPS),
                "dbscan_eps_m": eps_m,
                "dbscan_min_samples": min_samples,
                "poly_point_buffer_m": point_buffer_m,
                "poly_simplify_m": simplify_m,
            }
            poly_ll = None

        rows.append(row)
        if poly_ll is not None and len(poly_ll) > 0:
            polys.append(poly_ll)

        pd.DataFrame(rows).to_csv(OUT_CSV, index=False)
        print(f"  Progress saved: {OUT_CSV} ({len(rows)}/{len(CITIES)})")

        time.sleep(PAUSE_BETWEEN_CITIES_S)

    if polys:
        gdf_all = pd.concat(polys, ignore_index=True)
        gdf_all = gpd.GeoDataFrame(gdf_all, geometry="geometry", crs="EPSG:4326")

        try:
            if os.path.exists(OUT_GPKG):
                os.remove(OUT_GPKG)
        except Exception:
            pass
        gdf_all.to_file(OUT_GPKG, layer="business_areas", driver="GPKG")
        gdf_all.to_file(OUT_GEOJSON_ALL, driver="GeoJSON")

        print(f"\nSaved polygons: {OUT_GPKG}")
        print(f"Saved polygons: {OUT_GEOJSON_ALL}")

    print(f"\nSaved results CSV: {OUT_CSV}")

if __name__ == "__main__":
    main()


Resuming from data/city_centers\city_centers_business_polygon.csv. Cities already done: 3

Processing: Rome, Italy
  core radius km: 12.0
  dbscan eps/min_samples: 450.0/25
  polygon buffer/simplify (m): 180.0/60.0
  Progress saved: data/city_centers\city_centers_business_polygon.csv (4/94)

Processing: Madrid, Spain
  core radius km: 12.0
  dbscan eps/min_samples: 450.0/25
  polygon buffer/simplify (m): 180.0/60.0
  Progress saved: data/city_centers\city_centers_business_polygon.csv (5/94)

Processing: Lisbon, Portugal
  core radius km: 12.0
  dbscan eps/min_samples: 450.0/25
  polygon buffer/simplify (m): 180.0/60.0
  Progress saved: data/city_centers\city_centers_business_polygon.csv (6/94)

Processing: Amsterdam, Netherlands
  core radius km: 12.0
  dbscan eps/min_samples: 450.0/25
  polygon buffer/simplify (m): 180.0/60.0
  Progress saved: data/city_centers\city_centers_business_polygon.csv (7/94)

Processing: Brussels, Belgium
  core radius km: 12.0
  dbscan eps/min_samples: 450.

In [23]:
# plot_business_polygons_palette.py
# Make a "palette" (small multiples) of your business-area polygons
# No basemap, just shapes. Saves PNG (and optional PDF).
#
# Expects per-city polygons from:
#   data/city_centers/business_area_polygons_geojson/*.geojson
# Or a combined file:
#   data/city_centers/business_areas_100.geojson
#
# Usage:
#   python plot_business_polygons_palette.py
#
# Optional tweaks: see CONFIG section.

import os
import math
import glob
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=UserWarning)

# -------------------------
# CONFIG
# -------------------------
POLY_DIR = "data/city_centers/business_area_polygons_geojson"
COMBINED_GEOJSON = "data/city_centers/business_areas_100.geojson"

OUT_DIR = "data/city_centers/plots"
os.makedirs(OUT_DIR, exist_ok=True)

OUT_PNG = os.path.join(OUT_DIR, "business_polygons_palette.png")
OUT_PDF = os.path.join(OUT_DIR, "business_polygons_palette.pdf")

# Choose input mode: "dir" reads many per-city geojsons, "combined" reads one file
INPUT_MODE = "dir"   # "dir" or "combined"

# How many cities to plot (None = all)
MAX_CITIES = None

# Sorting: "area_desc", "area_asc", "name_asc"
SORT_BY = "area_desc"

# Layout
N_COLS = 10
FIG_W = 22
FIG_H = 18

# Normalize each polygon to its own local coordinate system for comparability
# "center_scale" plots each polygon centered at (0,0) and scaled by its max extent
# "center_only" centers but keeps original extents (in degrees if WGS84)
NORMALIZE = "center_scale"  # "center_scale" or "center_only" or "none"

# Drawing
LINEWIDTH = 1.0
SHOW_TITLE = True
TITLE_FIELD = "city"  # or "city (area_km2)" via code below

# Optionally annotate area
SHOW_AREA_TEXT = True

# If you want equal aspect in every panel (recommended)
FORCE_EQUAL_ASPECT = True

# Background
AXIS_OFF = True

# If you want to export multiple pages for many cities
MULTIPAGE = False
CITIES_PER_PAGE = 80

# -------------------------
# Helpers
# -------------------------
def load_polygons():
    if INPUT_MODE == "combined":
        if not os.path.exists(COMBINED_GEOJSON):
            raise FileNotFoundError(f"Missing {COMBINED_GEOJSON}")
        gdf = gpd.read_file(COMBINED_GEOJSON)
        return gdf

    files = sorted(glob.glob(os.path.join(POLY_DIR, "*.geojson")))
    if not files:
        raise FileNotFoundError(f"No GeoJSON files found in {POLY_DIR}")

    frames = []
    for fp in files:
        try:
            frames.append(gpd.read_file(fp))
        except Exception:
            pass
    if not frames:
        raise RuntimeError("Could not read any polygon files.")
    gdf = pd.concat(frames, ignore_index=True)
    gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs="EPSG:4326")
    return gdf

def ensure_area_km2(gdf):
    if "area_km2" in gdf.columns and gdf["area_km2"].notna().any():
        return gdf
    # compute approximate area from projected CRS (EPSG:3857) as fallback
    gdf_m = gdf.to_crs("EPSG:3857")
    gdf["area_km2"] = gdf_m.geometry.area / 1e6
    return gdf

def normalize_geom(geom, mode="center_scale"):
    if geom is None or geom.is_empty:
        return geom

    x, y = geom.exterior.coords.xy if geom.geom_type == "Polygon" else (None, None)

    # Use bounds for normalization
    minx, miny, maxx, maxy = geom.bounds
    cx = 0.5 * (minx + maxx)
    cy = 0.5 * (miny + maxy)

    def shift(g):
        return g.translate(-cx, -cy)

    # Shapely translate/scale via affinity
    try:
        from shapely import affinity
    except Exception:
        return geom

    g0 = affinity.translate(geom, xoff=-cx, yoff=-cy)

    if mode == "center_only":
        return g0

    if mode == "center_scale":
        dx = maxx - minx
        dy = maxy - miny
        scale = max(dx, dy)
        if not np.isfinite(scale) or scale == 0:
            return g0
        g1 = affinity.scale(g0, xfact=1.0/scale, yfact=1.0/scale, origin=(0, 0))
        return g1

    return geom

def prepare_gdf(gdf):
    gdf = gdf[gdf.geometry.notna()].copy()
    gdf = gdf[~gdf.geometry.is_empty].copy()

    if "city" not in gdf.columns:
        gdf["city"] = [f"city_{i}" for i in range(len(gdf))]

    gdf = ensure_area_km2(gdf)

    if SORT_BY == "area_desc":
        gdf = gdf.sort_values("area_km2", ascending=False)
    elif SORT_BY == "area_asc":
        gdf = gdf.sort_values("area_km2", ascending=True)
    elif SORT_BY == "name_asc":
        gdf = gdf.sort_values("city", ascending=True)

    if MAX_CITIES is not None:
        gdf = gdf.head(int(MAX_CITIES))

    return gdf.reset_index(drop=True)

def plot_page(gdf_page, page_idx=1, out_png=None, out_pdf=None):
    n = len(gdf_page)
    ncols = min(N_COLS, max(1, n))
    nrows = int(math.ceil(n / ncols))

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(FIG_W, FIG_H))
    if isinstance(axes, np.ndarray):
        axes = axes.flatten()
    else:
        axes = [axes]

    for ax in axes[n:]:
        ax.set_visible(False)

    for i in range(n):
        ax = axes[i]
        row = gdf_page.iloc[i]
        geom = row.geometry

        # Normalize for palette view
        if NORMALIZE in ("center_scale", "center_only"):
            geom_plot = normalize_geom(geom, mode=NORMALIZE)
        else:
            geom_plot = geom

        # Build a one-row GeoSeries for plotting
        gs = gpd.GeoSeries([geom_plot], crs=gdf_page.crs)

        gs.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=LINEWIDTH)

        if FORCE_EQUAL_ASPECT:
            ax.set_aspect("equal", adjustable="box")

        if AXIS_OFF:
            ax.set_axis_off()

        if SHOW_TITLE:
            name = str(row.get("city", ""))
            if SHOW_AREA_TEXT:
                a = row.get("area_km2", np.nan)
                if np.isfinite(a):
                    title = f"{name}\n{a:.2f} km²"
                else:
                    title = name
            else:
                title = name
            ax.set_title(title, fontsize=10)

        # Tight limits around the shape
        try:
            minx, miny, maxx, maxy = geom_plot.bounds
            pad = 0.08 * max(maxx - minx, maxy - miny, 1e-9)
            ax.set_xlim(minx - pad, maxx + pad)
            ax.set_ylim(miny - pad, maxy + pad)
        except Exception:
            pass

    fig.tight_layout()

    if out_png:
        fig.savefig(out_png, dpi=200, bbox_inches="tight")
    if out_pdf:
        fig.savefig(out_pdf, bbox_inches="tight")

    plt.close(fig)

def main():
    gdf = load_polygons()
    gdf = prepare_gdf(gdf)

    if not MULTIPAGE:
        plot_page(gdf, page_idx=1, out_png=OUT_PNG, out_pdf=OUT_PDF)
        print(f"Saved: {OUT_PNG}")
        print(f"Saved: {OUT_PDF}")
        return

    # Multipage mode: write separate PNGs
    total = len(gdf)
    pages = int(math.ceil(total / CITIES_PER_PAGE))
    for p in range(pages):
        s = p * CITIES_PER_PAGE
        e = min((p + 1) * CITIES_PER_PAGE, total)
        gdf_page = gdf.iloc[s:e].copy()
        out_png = os.path.join(OUT_DIR, f"business_polygons_palette_page{p+1}.png")
        plot_page(gdf_page, page_idx=p+1, out_png=out_png, out_pdf=None)
        print(f"Saved: {out_png}")

if __name__ == "__main__":
    main()


Saved: data/city_centers/plots\business_polygons_palette.png
Saved: data/city_centers/plots\business_polygons_palette.pdf


In [26]:
# folium_business_areas_with_city_borders.py
# Overlay business-area polygons + real city borders (OSM) in Folium.
#
# Inputs (pick one):
# 1) Combined polygons: data/city_centers/business_areas_100.geojson
# 2) Per-city polygons: data/city_centers/business_area_polygons_geojson/*.geojson
#
# Outputs:
# - data/city_centers/maps/business_areas_with_borders_all.html
# - optionally, per-city HTML maps in data/city_centers/maps/per_city/

import os
import re
import json
import glob
import warnings
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium

from shapely.ops import unary_union

warnings.filterwarnings("ignore", category=UserWarning)

# -------------------------
# CONFIG
# -------------------------
INPUT_MODE = "combined"  # "combined" or "dir"
COMBINED_GEOJSON = "data/city_centers/city_centers_business_polygon.geojson"
POLY_DIR = "data/city_centers/business_area_polygons_geojson"

OUT_DIR = "data/city_centers/maps"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_HTML_ALL = os.path.join(OUT_DIR, "business_areas_with_borders_all.html")

MAKE_PER_CITY = True
PER_CITY_DIR = os.path.join(OUT_DIR, "per_city")
os.makedirs(PER_CITY_DIR, exist_ok=True)

# City border style
BORDER_COLOR = "#1f77b4"
BORDER_WEIGHT = 2
BORDER_OPACITY = 0.9
BORDER_FILL_OPACITY = 0.05

# Business polygon style
BUSINESS_COLOR = "#d62728"
BUSINESS_WEIGHT = 2
BUSINESS_OPACITY = 0.95
BUSINESS_FILL_OPACITY = 0.30

# Map behavior
TILES = "CartoDB positron"
START_ZOOM = 2  # world view for combined map

# -------------------------
# Helpers
# -------------------------
def slugify(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[^a-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s[:120]

def configure_osmnx():
    ox.settings.use_cache = True
    ox.settings.log_console = True
    ox.settings.timeout = 180
    if hasattr(ox.settings, "overpass_rate_limit"):
        ox.settings.overpass_rate_limit = True

def load_business_polygons():
    if INPUT_MODE == "combined":
        if not os.path.exists(COMBINED_GEOJSON):
            raise FileNotFoundError(f"Missing: {COMBINED_GEOJSON}")
        gdf = gpd.read_file(COMBINED_GEOJSON)
        if gdf.crs is None:
            gdf = gdf.set_crs("EPSG:4326")
        else:
            gdf = gdf.to_crs("EPSG:4326")
        return gdf

    files = sorted(glob.glob(os.path.join(POLY_DIR, "*.geojson")))
    if not files:
        raise FileNotFoundError(f"No GeoJSON files found in {POLY_DIR}")
    frames = []
    for fp in files:
        try:
            frames.append(gpd.read_file(fp))
        except Exception:
            pass
    if not frames:
        raise RuntimeError("Could not read any polygon files.")
    gdf = pd.concat(frames, ignore_index=True)
    gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs="EPSG:4326")
    gdf = gdf.to_crs("EPSG:4326")
    return gdf

def get_city_boundary_ll(city_query: str) -> gpd.GeoDataFrame:
    gdf = ox.geocode_to_gdf(city_query)
    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
    if gdf.empty:
        raise RuntimeError(f"Could not fetch polygon for {city_query}")
    geom = gdf.geometry.iloc[0]
    try:
        geom = geom.union_all()
    except Exception:
        pass
    out = gpd.GeoDataFrame({"city": [city_query]}, geometry=[geom], crs=gdf.crs)
    return out.to_crs("EPSG:4326")

def centroid_latlon(geom):
    c = geom.centroid
    return float(c.y), float(c.x)

def add_geojson_layer(m, gdf_ll, name, color, weight, opacity, fill_opacity, tooltip_fields=None):
    if gdf_ll.empty:
        return None

    tooltip = None
    if tooltip_fields:
        tooltip = folium.GeoJsonTooltip(fields=tooltip_fields, sticky=True)

    layer = folium.GeoJson(
        data=json.loads(gdf_ll.to_json()),
        name=name,
        style_function=lambda feat: {
            "color": color,
            "weight": weight,
            "opacity": opacity,
            "fillColor": color,
            "fillOpacity": fill_opacity,
        },
        tooltip=tooltip,
    )
    layer.add_to(m)
    return layer

def fit_map_to_bounds(m, geom):
    minx, miny, maxx, maxy = geom.bounds
    m.fit_bounds([[miny, minx], [maxy, maxx]])

# -------------------------
# Main
# -------------------------
def main():
    configure_osmnx()

    biz = load_business_polygons()
    if "city" not in biz.columns:
        raise RuntimeError("Your business polygons file(s) must have a 'city' column.")

    biz = biz[biz.geometry.notna() & ~biz.geometry.is_empty].copy()

    # Combined map
    m_all = folium.Map(location=[20, 0], zoom_start=START_ZOOM, tiles=TILES)

    fg_borders = folium.FeatureGroup(name="City borders", show=True).add_to(m_all)
    fg_business = folium.FeatureGroup(name="Business areas", show=True).add_to(m_all)

    boundaries_cache = {}

    for i, city in enumerate(biz["city"].astype(str).tolist(), start=1):
        print(f"Processing {i}/{len(biz)}: {city}")

        # business polygon row
        row = biz[biz["city"] == city].iloc[0]
        biz_gdf = gpd.GeoDataFrame([row.drop(labels=["geometry"])], geometry=[row.geometry], crs="EPSG:4326")

        # fetch boundary (cache)
        if city not in boundaries_cache:
            try:
                boundaries_cache[city] = get_city_boundary_ll(city)
            except Exception as e:
                print(f"  Boundary fetch failed for {city}: {e}")
                boundaries_cache[city] = gpd.GeoDataFrame({"city": [city]}, geometry=[None], crs="EPSG:4326")

        bnd = boundaries_cache[city]
        bnd_geom = bnd.geometry.iloc[0] if (not bnd.empty and bnd.geometry.notna().any()) else None

        # Add border
        if bnd_geom is not None and not bnd_geom.is_empty:
            add_geojson_layer(
                fg_borders, bnd, name=f"Border: {city}",
                color=BORDER_COLOR, weight=BORDER_WEIGHT,
                opacity=BORDER_OPACITY, fill_opacity=BORDER_FILL_OPACITY,
                tooltip_fields=["city"],
            )

        # Add business polygon
        tooltip_fields = [c for c in ["city", "area_km2", "business_points_total", "business_points_cluster", "cluster_method"] if c in biz_gdf.columns]
        add_geojson_layer(
            fg_business, biz_gdf, name=f"Business: {city}",
            color=BUSINESS_COLOR, weight=BUSINESS_WEIGHT,
            opacity=BUSINESS_OPACITY, fill_opacity=BUSINESS_FILL_OPACITY,
            tooltip_fields=tooltip_fields if tooltip_fields else None,
        )

        # Optional per-city map
        if MAKE_PER_CITY:
            # center the city map around business polygon if possible, else border
            center_geom = row.geometry if row.geometry is not None else bnd_geom
            if center_geom is not None and not center_geom.is_empty:
                lat, lon = centroid_latlon(center_geom)
                m_city = folium.Map(location=[lat, lon], zoom_start=12, tiles=TILES)
            else:
                m_city = folium.Map(location=[0, 0], zoom_start=2, tiles=TILES)

            fg1 = folium.FeatureGroup(name="City border", show=True).add_to(m_city)
            fg2 = folium.FeatureGroup(name="Business area", show=True).add_to(m_city)

            if bnd_geom is not None and not bnd_geom.is_empty:
                add_geojson_layer(
                    fg1, bnd, name="Border",
                    color=BORDER_COLOR, weight=BORDER_WEIGHT,
                    opacity=BORDER_OPACITY, fill_opacity=BORDER_FILL_OPACITY,
                    tooltip_fields=["city"],
                )

            add_geojson_layer(
                fg2, biz_gdf, name="Business area",
                color=BUSINESS_COLOR, weight=BUSINESS_WEIGHT,
                opacity=BUSINESS_OPACITY, fill_opacity=BUSINESS_FILL_OPACITY,
                tooltip_fields=tooltip_fields if tooltip_fields else None,
            )

            folium.LayerControl(collapsed=True).add_to(m_city)

            # Fit bounds to whichever geometry exists
            if bnd_geom is not None and not bnd_geom.is_empty:
                fit_map_to_bounds(m_city, bnd_geom)
            elif row.geometry is not None and not row.geometry.is_empty:
                fit_map_to_bounds(m_city, row.geometry)

            out_city = os.path.join(PER_CITY_DIR, f"{slugify(city)}.html")
            m_city.save(out_city)

    folium.LayerControl(collapsed=True).add_to(m_all)
    m_all.save(OUT_HTML_ALL)

    print(f"\nSaved combined map: {OUT_HTML_ALL}")
    if MAKE_PER_CITY:
        print(f"Saved per-city maps in: {PER_CITY_DIR}")

if __name__ == "__main__":
    main()


FileNotFoundError: Missing: data/city_centers/city_centers_business_polygon.geojson

In [15]:
# moscow_dbscan_tuning_folium.py
# Folium map to tune DBSCAN settings for Moscow business area polygon.
#
# Shows:
# 1) Moscow administrative boundary (from OSM geocode polygon)
# 2) Business points (as a HeatMap layer)
# 3) Largest DBSCAN cluster points (optional, downsampled)
# 4) Business area polygon built from buffered union of cluster points
#
# Change the parameters in the CONFIG block and rerun to compare results.
#
# Output:
#   data/city_centers/maps/moscow_dbscan_tuning.html

import os
import math
import random
import warnings
import numpy as np
import geopandas as gpd
import osmnx as ox
import folium
from folium.plugins import HeatMap, FastMarkerCluster
from shapely.geometry import Point
from shapely.ops import unary_union

warnings.filterwarnings("ignore", category=UserWarning)

# -------------------------
# CONFIG (edit these by hand)
# -------------------------
CITY_QUERY = "Moscow, Russia"
HISTORIC_QUERY = "Red Square, Moscow, Russia"   # used as map center and optional core circle center

# Data fetch region:
# Using full admin boundary for fetching is often too heavy.
# We fetch within a core circle but still display the admin boundary.
FETCH_RADIUS_KM = 10.0

# Business tags (OSM)
INCLUDE_SHOPS = True

# DBSCAN params (meters, after projecting to UTM)
DBSCAN_EPS_M = 420
DBSCAN_MIN_SAMPLES = 60

# Polygon construction params (meters, UTM)
POLY_POINT_BUFFER_M = 140
POLY_SIMPLIFY_M = 80  # set 0 to disable simplify

# Points display controls
MAX_POINTS_HEATMAP = 100        # downsample for heatmap if too many points
MAX_POINTS_CLUSTER_LAYER = 80   # downsample for cluster layer

# Output
OUT_DIR = "data/city_centers/maps"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_HTML = os.path.join(OUT_DIR, "moscow_dbscan_tuning.html")

# -------------------------
# Helpers
# -------------------------
def configure_osmnx():
    ox.settings.use_cache = True
    ox.settings.log_console = True
    ox.settings.timeout = 180
    if hasattr(ox.settings, "overpass_rate_limit"):
        ox.settings.overpass_rate_limit = True

def business_tags(include_shops: bool):
    tags = {
        "office": True,
        "building": ["office", "commercial"],
        "landuse": ["commercial", "retail"],
        "amenity": ["bank", "conference_centre"],
    }
    if include_shops:
        tags["shop"] = True
    return tags

def safe_geocode_point(query: str) -> Point:
    lat, lon = ox.geocode(query)  # returns (lat, lon)
    return Point(float(lon), float(lat))

def get_city_boundary_ll(city_query: str) -> gpd.GeoDataFrame:
    gdf = ox.geocode_to_gdf(city_query)
    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
    if gdf.empty:
        raise RuntimeError(f"Could not fetch boundary for {city_query}")
    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326")
    else:
        gdf = gdf.to_crs("EPSG:4326")
    # Keep first geometry
    out = gpd.GeoDataFrame({"city": [city_query]}, geometry=[gdf.geometry.iloc[0]], crs="EPSG:4326")
    return out

def choose_local_utm_crs(boundary_ll: gpd.GeoDataFrame) -> str:
    try:
        crs = boundary_ll.estimate_utm_crs()
        if crs is not None:
            return crs.to_string()
    except Exception:
        pass
    # fallback
    c = boundary_ll.geometry.iloc[0].centroid
    lon = float(c.x)
    lat = float(c.y)
    zone = int((lon + 180.0) // 6) + 1
    epsg = (32600 + zone) if (lat >= 0) else (32700 + zone)
    return f"EPSG:{epsg}"

def fetch_features(poly_ll, tags):
    # compatible with different OSMnx versions
    try:
        return ox.features_from_polygon(poly_ll, tags)
    except Exception:
        return ox.geometries_from_polygon(poly_ll, tags)

def features_to_points_ll(gdf_ll: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    if gdf_ll is None or len(gdf_ll) == 0:
        return gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

    def to_pt(g):
        if g is None:
            return None
        if g.geom_type == "Point":
            return g
        try:
            return g.representative_point()
        except Exception:
            return g.centroid

    pts = gdf_ll.copy()
    pts["geometry"] = pts.geometry.apply(to_pt)
    pts = pts[pts.geometry.notna()].copy()
    pts = pts.set_geometry("geometry")
    if pts.crs is None:
        pts = pts.set_crs("EPSG:4326")
    else:
        pts = pts.to_crs("EPSG:4326")
    pts = pts[pts.geometry.type == "Point"].copy()
    return pts[["geometry"]].copy()

def run_dbscan(points_xy: np.ndarray, eps_m: float, min_samples: int):
    from sklearn.cluster import DBSCAN
    model = DBSCAN(eps=float(eps_m), min_samples=int(min_samples))
    return model.fit_predict(points_xy)

def largest_polygon(geom):
    if geom is None or geom.is_empty:
        return None
    if geom.geom_type == "Polygon":
        return geom
    if geom.geom_type == "MultiPolygon":
        return max(list(geom.geoms), key=lambda g: g.area)
    return None

def downsample_latlon(latlons, max_n):
    if len(latlons) <= max_n:
        return latlons
    idx = np.random.choice(len(latlons), size=max_n, replace=False)
    return [latlons[i] for i in idx]

# -------------------------
# Main build
# -------------------------
def make_map():
    configure_osmnx()

    # Admin boundary
    boundary_ll = get_city_boundary_ll(CITY_QUERY)

    # Map center at historic point
    g_ll = safe_geocode_point(HISTORIC_QUERY)
    center_lat = float(g_ll.y)
    center_lon = float(g_ll.x)

    # Choose local metric CRS
    utm_crs = choose_local_utm_crs(boundary_ll)

    # Fetch polygon: core circle in UTM then back to WGS84 for Overpass query
    g_utm = gpd.GeoDataFrame(geometry=[g_ll], crs="EPSG:4326").to_crs(utm_crs).geometry.iloc[0]
    core_circle_utm = g_utm.buffer(FETCH_RADIUS_KM * 1000.0)

    boundary_utm = boundary_ll.to_crs(utm_crs)
    core_utm = boundary_utm.geometry.iloc[0].intersection(core_circle_utm)
    if core_utm.is_empty:
        core_utm = core_circle_utm

    core_ll = gpd.GeoDataFrame({"name": ["core"]}, geometry=[core_utm], crs=utm_crs).to_crs("EPSG:4326")
    core_poly_ll = core_ll.geometry.iloc[0]

    # Fetch business features within core region
    tags = business_tags(INCLUDE_SHOPS)
    feat_ll = fetch_features(core_poly_ll, tags)
    if feat_ll is None or len(feat_ll) == 0:
        raise RuntimeError("No business features returned. Try INCLUDE_SHOPS=True or increase FETCH_RADIUS_KM.")

    if feat_ll.crs is None:
        feat_ll = feat_ll.set_crs("EPSG:4326")
    else:
        feat_ll = feat_ll.to_crs("EPSG:4326")

    pts_ll = features_to_points_ll(feat_ll)
    n_total = len(pts_ll)
    print(f"Business points total: {n_total}")

    pts_utm = pts_ll.to_crs(utm_crs)
    xy = np.column_stack([pts_utm.geometry.x.values, pts_utm.geometry.y.values])

    # DBSCAN
    labels = run_dbscan(xy, DBSCAN_EPS_M, DBSCAN_MIN_SAMPLES)

    # Choose largest cluster (ignore noise -1)
    valid = labels[labels >= 0]
    if len(valid) == 0:
        print("DBSCAN produced only noise. Try bigger DBSCAN_EPS_M or smaller DBSCAN_MIN_SAMPLES.")
        cluster_pts_utm = pts_utm.iloc[0:0].copy()
        cluster_method = "dbscan_all_noise"
    else:
        uniq, cnt = np.unique(valid, return_counts=True)
        best_label = int(uniq[np.argmax(cnt)])
        cluster_pts_utm = pts_utm.iloc[labels == best_label].copy()
        cluster_method = f"dbscan_eps{DBSCAN_EPS_M}_min{DBSCAN_MIN_SAMPLES}_label{best_label}"
        print(f"Cluster points: {len(cluster_pts_utm)} (method: {cluster_method})")

    # Business polygon from cluster points
    poly_utm = None
    area_km2 = 0.0
    if len(cluster_pts_utm) > 0:
        bufs = cluster_pts_utm.geometry.buffer(POLY_POINT_BUFFER_M)
        merged = unary_union(bufs.values)
        poly_utm = largest_polygon(merged)
        if poly_utm is not None and not poly_utm.is_empty:
            if POLY_SIMPLIFY_M and POLY_SIMPLIFY_M > 0:
                try:
                    poly_utm = poly_utm.simplify(POLY_SIMPLIFY_M, preserve_topology=True)
                except Exception:
                    pass
            area_km2 = float(poly_utm.area) / 1e6

    # Convert polygon to WGS84
    poly_ll_gdf = gpd.GeoDataFrame(
        {
            "city": [CITY_QUERY],
            "area_km2": [area_km2],
            "cluster_method": [cluster_method],
            "eps_m": [DBSCAN_EPS_M],
            "min_samples": [DBSCAN_MIN_SAMPLES],
            "point_buffer_m": [POLY_POINT_BUFFER_M],
            "simplify_m": [POLY_SIMPLIFY_M],
            "fetch_radius_km": [FETCH_RADIUS_KM],
            "points_total": [n_total],
            "points_cluster": [int(len(cluster_pts_utm))],
        },
        geometry=[poly_utm] if poly_utm is not None else [None],
        crs=utm_crs,
    ).to_crs("EPSG:4326")

    # Prepare Folium map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles="CartoDB positron")

    # Admin boundary layer
    folium.GeoJson(
        data=json.loads(boundary_ll.to_json()),
        name="Moscow administrative boundary",
        style_function=lambda f: {
            "color": "#1f77b4",
            "weight": 2,
            "opacity": 0.9,
            "fillColor": "#1f77b4",
            "fillOpacity": 0.05,
        },
        tooltip=folium.GeoJsonTooltip(fields=["city"], sticky=True),
    ).add_to(m)

    # Core fetch region layer
    folium.GeoJson(
        data=json.loads(core_ll.to_json()),
        name=f"Core fetch circle (intersected) {FETCH_RADIUS_KM} km",
        style_function=lambda f: {
            "color": "#7f7f7f",
            "weight": 2,
            "opacity": 0.9,
            "fillColor": "#7f7f7f",
            "fillOpacity": 0.03,
        },
        tooltip="Core fetch region",
    ).add_to(m)

    # Historic center marker
    folium.Marker(
        location=[center_lat, center_lon],
        popup=HISTORIC_QUERY,
        tooltip="Historic center (map center)",
    ).add_to(m)

    # Heatmap of all business points (downsampled)
    all_latlons = [(float(p.y), float(p.x)) for p in pts_ll.geometry.values]
    all_latlons = downsample_latlon(all_latlons, MAX_POINTS_HEATMAP)
    HeatMap(all_latlons, name="Business points heatmap", radius=8, blur=12, max_zoom=13).add_to(m)

    # Optional cluster points layer (downsampled)
    if len(cluster_pts_utm) > 0:
        cluster_ll = cluster_pts_utm.to_crs("EPSG:4326")
        cluster_latlons = [(float(p.y), float(p.x)) for p in cluster_ll.geometry.values]
        cluster_latlons = downsample_latlon(cluster_latlons, MAX_POINTS_CLUSTER_LAYER)
        FastMarkerCluster(cluster_latlons, name="Largest DBSCAN cluster points").add_to(m)

    # Business polygon layer
    if poly_ll_gdf.geometry.notna().any() and (not poly_ll_gdf.geometry.iloc[0].is_empty):
        folium.GeoJson(
            data=json.loads(poly_ll_gdf.to_json()),
            name=f"Business area polygon (area {area_km2:.2f} km2)",
            style_function=lambda f: {
                "color": "#d62728",
                "weight": 2,
                "opacity": 0.95,
                "fillColor": "#d62728",
                "fillOpacity": 0.30,
            },
            tooltip=folium.GeoJsonTooltip(
                fields=["city", "area_km2", "cluster_method", "eps_m", "min_samples", "point_buffer_m", "fetch_radius_km", "points_total", "points_cluster"],
                sticky=True,
            ),
        ).add_to(m)
    else:
        folium.map.Marker(
            [center_lat, center_lon],
            icon=folium.DivIcon(html="<div style='font-size:12px;color:#d62728;'>No polygon (try different DBSCAN settings)</div>")
        ).add_to(m)

    folium.LayerControl(collapsed=False).add_to(m)

    # Fit map to boundary
    minx, miny, maxx, maxy = boundary_ll.geometry.iloc[0].bounds
    m.fit_bounds([[miny, minx], [maxy, maxx]])

    m.save(OUT_HTML)
    print(f"Saved: {OUT_HTML}")

if __name__ == "__main__":
    make_map()


Business points total: 35437
Cluster points: 24156 (method: dbscan_eps420_min60_label0)
Saved: data/city_centers/maps\moscow_dbscan_tuning.html


In [28]:
import os
import json
import math
import argparse
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import unary_union


def utm_epsg_from_lonlat(lon: float, lat: float) -> str:
    zone = int((lon + 180.0) // 6) + 1
    epsg = (32600 + zone) if (lat >= 0) else (32700 + zone)
    return f"EPSG:{epsg}"


def choose_local_metric_crs(g_lon: float, g_lat: float, poly_ll: gpd.GeoDataFrame | None) -> str:
    # Try estimate_utm_crs from polygon if present, else use lon/lat
    if poly_ll is not None and not poly_ll.empty:
        try:
            crs = poly_ll.estimate_utm_crs()
            if crs is not None:
                return crs.to_string()
        except Exception:
            pass
    return utm_epsg_from_lonlat(g_lon, g_lat)


def largest_polygon(geom):
    if geom is None or geom.is_empty:
        return None
    gt = geom.geom_type
    if gt == "Polygon":
        return geom
    if gt == "MultiPolygon":
        return max(list(geom.geoms), key=lambda g: g.area)
    return None


def polygon_to_rel_ring_xy(poly_metric, g_metric_point: Point, simplify_m: float):
    poly = largest_polygon(poly_metric)
    if poly is None:
        return None

    if simplify_m and simplify_m > 0:
        try:
            poly = poly.simplify(simplify_m, preserve_topology=True)
        except Exception:
            pass

    ring = list(poly.exterior.coords)
    gx, gy = float(g_metric_point.x), float(g_metric_point.y)
    rel = [[float(x - gx), float(y - gy)] for (x, y) in ring]
    return rel


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv", required=True, help="data/city_centers/city_centers_business_polygon.csv")
    ap.add_argument("--out", required=True, help="output json path, e.g. data/ui/cities_static.json")
    ap.add_argument("--simplify_m", type=float, default=80.0)
    args = ap.parse_args()

    df = pd.read_csv(args.csv)
    needed = {"city", "historic_lon", "historic_lat", "business_lon", "business_lat", "business_area_km2", "polygon_file"}
    missing = needed - set(df.columns)
    if missing:
        raise RuntimeError(f"CSV missing columns: {sorted(missing)}")

    items = []
    extent_m = 0.0

    for _, r in df.iterrows():
        city = str(r["city"])
        g_lon = float(r["historic_lon"]) if pd.notna(r["historic_lon"]) else None
        g_lat = float(r["historic_lat"]) if pd.notna(r["historic_lat"]) else None
        mu_lon = float(r["business_lon"]) if pd.notna(r["business_lon"]) else None
        mu_lat = float(r["business_lat"]) if pd.notna(r["business_lat"]) else None
        area_km2 = float(r["business_area_km2"]) if pd.notna(r["business_area_km2"]) else None
        poly_fp = str(r["polygon_file"]) if pd.notna(r["polygon_file"]) else ""

        if g_lon is None or g_lat is None or mu_lon is None or mu_lat is None:
            continue

        poly_ll = None
        if poly_fp and os.path.exists(poly_fp) and os.path.getsize(poly_fp) > 0:
            try:
                poly_ll = gpd.read_file(poly_fp)
                if poly_ll.crs is None:
                    poly_ll = poly_ll.set_crs("EPSG:4326")
                else:
                    poly_ll = poly_ll.to_crs("EPSG:4326")
            except Exception:
                poly_ll = None

        metric_crs = choose_local_metric_crs(g_lon, g_lat, poly_ll)

        g_metric = gpd.GeoSeries([Point(g_lon, g_lat)], crs="EPSG:4326").to_crs(metric_crs).iloc[0]
        mu_metric = gpd.GeoSeries([Point(mu_lon, mu_lat)], crs="EPSG:4326").to_crs(metric_crs).iloc[0]
        mu_rel = [float(mu_metric.x - g_metric.x), float(mu_metric.y - g_metric.y)]

        ring_rel = None
        if poly_ll is not None and not poly_ll.empty:
            poly_metric = poly_ll.to_crs(metric_crs).geometry.iloc[0]
            ring_rel = polygon_to_rel_ring_xy(poly_metric, g_metric, simplify_m=args.simplify_m)

        # Update extent
        extent_m = max(extent_m, abs(mu_rel[0]), abs(mu_rel[1]))
        if ring_rel:
            for x, y in ring_rel:
                extent_m = max(extent_m, abs(x), abs(y))

        items.append(
            {
                "city": city,
                "area_km2": area_km2,
                "mu_rel": mu_rel,        # meters, relative to historic center
                "poly_rel": ring_rel,    # meters, relative to historic center (outer ring)
            }
        )

    payload = {
        "meta": {
            # pad a bit so blobs do not touch the frame
            "extent_m": float(extent_m * 1.15) if extent_m > 0 else 10000.0
        },
        "cities": items,
    }

    os.makedirs(os.path.dirname(args.out), exist_ok=True)
    with open(args.out, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False)

    print(f"Saved: {args.out}")
    print(f"Cities: {len(items)} | extent_m: {payload['meta']['extent_m']:.1f}")


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] --csv CSV --out OUT [--simplify_m SIMPLIFY_M]
ipykernel_launcher.py: error: the following arguments are required: --csv, --out


SystemExit: 2